In [1]:
sentences = ["VADER is smart, handsome, and funny.",  # positive sentence example
             "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
             "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
             "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
             "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
             "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
             "VADER is not smart, handsome, nor funny.",  # negation sentence example
             "The book was good.",  # positive sentence
             "At least it isn't a horrible book.",  # negated negative sentence with contraction
             "The book was only kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
             "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
             "Today SUX!",  # negative slang with capitalization emphasis
             "Today only kinda sux! But I'll get by, lol", # mixed sentiment example with slang and constrastive conjunction "but"
             "Make sure you :) or :D today!",  # emoticons handled
             "Catch utf-8 emoji such as such as 💘 and 💋 and 😁",  # emojis handled
             "Not bad at all"  # Capitalized negation
             ]

## VaderSentiment

https://github.com/cjhutto/vaderSentiment

In [51]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [85]:
def get_vader_sentiment(text):
    vs = analyzer.polarity_scores(s).pop('compound', None)
    if vs > 0.05:
        return 'POS'
    
    elif vs < -0.05:
        return 'NEG'
    
    return 'NEU'

In [88]:
for s in sentences:
    print(get_vader_sentiment(s),":", s)

POS : VADER is smart, handsome, and funny.
POS : VADER is smart, handsome, and funny!
POS : VADER is very smart, handsome, and funny.
POS : VADER is VERY SMART, handsome, and FUNNY.
POS : VADER is VERY SMART, handsome, and FUNNY!!!
POS : VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!
NEG : VADER is not smart, handsome, nor funny.
POS : The book was good.
POS : At least it isn't a horrible book.
POS : The book was only kind of good.
NEG : The plot was good, but the characters are uncompelling and the dialog is not great.
NEG : Today SUX!
POS : Today only kinda sux! But I'll get by, lol
POS : Make sure you :) or :D today!
POS : Catch utf-8 emoji such as such as 💘 and 💋 and 😁
POS : Not bad at all


## Transformers

https://github.com/cardiffnlp/tweeteval

In [78]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

MODELPATH = f"cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODELPATH)
model = AutoModelForSequenceClassification.from_pretrained(MODELPATH)
model.save_pretrained(MODELPATH)
tokenizer.save_pretrained(MODELPATH)

('cardiffnlp/twitter-roberta-base-sentiment/tokenizer_config.json',
 'cardiffnlp/twitter-roberta-base-sentiment/special_tokens_map.json',
 'cardiffnlp/twitter-roberta-base-sentiment/vocab.json',
 'cardiffnlp/twitter-roberta-base-sentiment/merges.txt',
 'cardiffnlp/twitter-roberta-base-sentiment/added_tokens.json',
 'cardiffnlp/twitter-roberta-base-sentiment/tokenizer.json')

In [79]:
#https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt
mapping = {0: 'NEG', 1: 'NEU', 2:'POS'}

In [80]:
def get_transformers_sentiment(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1][0]
    result = mapping[ranking]
    
    return result

In [89]:
for s in sentences:
    print(get_transformers_sentiment(s),":", s)

POS : VADER is smart, handsome, and funny.
POS : VADER is smart, handsome, and funny!
POS : VADER is very smart, handsome, and funny.
POS : VADER is VERY SMART, handsome, and FUNNY.
POS : VADER is VERY SMART, handsome, and FUNNY!!!
POS : VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!
NEG : VADER is not smart, handsome, nor funny.
POS : The book was good.
NEU : At least it isn't a horrible book.
POS : The book was only kind of good.
NEG : The plot was good, but the characters are uncompelling and the dialog is not great.
NEU : Today SUX!
NEU : Today only kinda sux! But I'll get by, lol
POS : Make sure you :) or :D today!
NEU : Catch utf-8 emoji such as such as 💘 and 💋 and 😁
POS : Not bad at all


## Comparison

In [90]:
for s in sentences:
    v = get_vader_sentiment(s)
    t = get_transformers_sentiment(s)
    print("Matching? ", v == t, "      Sentence: ", s)
    print("Vader: ", v, "     ", "Transformers: ", t, "\n")

Matching?  True       Sentence:  VADER is smart, handsome, and funny.
Vader:  POS       Transformers:  POS 

Matching?  True       Sentence:  VADER is smart, handsome, and funny!
Vader:  POS       Transformers:  POS 

Matching?  True       Sentence:  VADER is very smart, handsome, and funny.
Vader:  POS       Transformers:  POS 

Matching?  True       Sentence:  VADER is VERY SMART, handsome, and FUNNY.
Vader:  POS       Transformers:  POS 

Matching?  True       Sentence:  VADER is VERY SMART, handsome, and FUNNY!!!
Vader:  POS       Transformers:  POS 

Matching?  True       Sentence:  VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!
Vader:  POS       Transformers:  POS 

Matching?  True       Sentence:  VADER is not smart, handsome, nor funny.
Vader:  NEG       Transformers:  NEG 

Matching?  True       Sentence:  The book was good.
Vader:  POS       Transformers:  POS 

Matching?  False       Sentence:  At least it isn't a horrible book.
Vader:  POS       Transformers:  NEU